In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import pickle

## Finding composition with Enrichment and Burnup
Testing time for various methods

In [24]:
enr = np.random.uniform(low=3, high=5, size=(10,))
br = np.random.uniform(low=30000, high=50000, size=(10,))
enr_br = np.column_stack((enr,br))
enr_br

array([[4.58007581e+00, 3.25606737e+04],
       [3.48187762e+00, 4.09312647e+04],
       [4.24827521e+00, 4.12782044e+04],
       [3.26541815e+00, 3.79421420e+04],
       [4.15393358e+00, 4.22326948e+04],
       [3.69767907e+00, 4.58861462e+04],
       [4.97473806e+00, 4.90708982e+04],
       [4.90771987e+00, 4.14164970e+04],
       [4.29812474e+00, 3.60585732e+04],
       [3.95642144e+00, 4.40485592e+04]])

## Pickle and model

In [3]:
# load pickled dictionary with
# key: isotope
# value: trained model
dep_dict = pickle.load(open('lin_dep.pkl',"rb"))

In [30]:
# get time for rom depletion for one assembly
# given enrichment and burnup value,
# predict UNF composition
%time

def get_a(f):
    z = ''
    for i in f:
        if i.isdigit():
            z += i
    return int(z)

depleted_composition_rom = {}
for indx, enr_br_set in enumerate(enr_br):
    depleted_composition_rom[indx] = {}
    x = [enr_br_set]
    for iso, alg in dep_dict.items():
        if 'alg' in iso:
            continue
        else:
            if any(i.isdigit() for i in dep_dict[iso + '_alg']):
                deg = get_a(dep_dict[iso + '_alg'])
                poly = sklearn.preprocessing.PolynomialFeatures(degree=deg)
                depleted_composition_rom[indx][iso] = dep_dict[iso].predict(poly.fit_transform(x))[0]
            else:
                depleted_composition_rom[indx][iso] = dep_dict[iso].predict(x)[0]
    
print('\n')
depleted_composition_rom



CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs




{0: {'zr-95': 1.4141053999968489e-06,
  'zr-93': 0.0008238252855731517,
  'ra-226': 7.367958638866163e-14,
  'ru-103': 6.835870566132582e-08,
  'ru-106': 6.704006269306254e-05,
  'sb-124': 6.096951573097028e-10,
  'sb-125': 8.33894732558593e-06,
  'pu-238': 0.00017196717608969873,
  'pu-239': 0.006923517764753972,
  'np-237': 0.0005252056610331217,
  'nb-95': 1.6628007134614637e-06,
  'pa-231': 6.800824390605832e-10,
  'pd-107': 0.0001881229142400308,
  'pm-147': 0.00015554271408988038,
  'pu-240': 0.0020688594462369386,
  'pu-241': 0.0014527464991619464,
  'pu-244': 1.3073438045276659e-08,
  'pu-242': 0.00033956997659996714,
  'i-129': 0.00015158554782001364,
  'h-3': 5.059568289509579e-08,
  'kr-85': 2.6964987235862302e-05,
  'nb-94': 1.3201228292784423e-09,
  'cm-248': 1.3684503249840266e-10,
  'cs-134': 8.011210754562629e-05,
  'cs-135': 0.0005632369440135467,
  'cm-245': 1.357317929527052e-06,
  'cm-244': 1.8203667856271183e-05,
  'cm-246': 8.489457592297309e-08,
  'cm-247': 1.386

## Looking up from db

In [27]:

df = pd.read_csv('./curated.csv', index_col=0)

In [29]:
%time
# get time for `look-up' method for one assembly
# find closest assembly to this burnup and enrichment
depleted_composition_lookup = {}
for indx, enr_br_set in enumerate(enr_br):
    df['err'] = abs(df['bu'] - enr_br[0][1]) + abs(df['init_enr'] - enr_br[0][0]) * 10000
    minimum_err = df['err'].argmin()
    df = df.drop(columns='err')
    average_assem = df.loc[minimum_err]
    depleted_composition_lookup[indx] = average_assem.iloc[5:].to_dict()

print('\n')
print(depleted_composition_lookup)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys




{0: {'zr-95': 1.0312124915801478e-06, 'zr-93': 0.0008280920446280762, 'ra-226': 8.201941169455579e-14, 'ru-103': 4.984640964222101e-08, 'ru-106': 5.8334375129421044e-05, 'sb-124': 4.58534114389526e-10, 'sb-125': 7.739466382266098e-06, 'pu-238': 0.00015499551162574524, 'pu-239': 0.006658304279596675, 'np-237': 0.0004891523386067885, 'nb-95': 1.216297470352895e-06, 'pa-231': 7.480152302016811e-10, 'pd-107': 0.00017940294322276498, 'pm-147': 0.00015047033173697672, 'pu-240': 0.0019938224163238622, 'pu-241': 0.0012804587062745733, 'pu-244': 1.2209439510696284e-08, 'pu-242': 0.00032550132458576865, 'i-129': 0.00014957749856162305, 'h-3': 5.0118209854536736e-08, 'kr-85': 3.0482092616173022e-05, 'nb-94': 1.1525537808330354e-09, 'cm-248': 2.9030855921804964e-11, 'cs-134': 7.524760620481831e-05, 'cs-135': 0.0006114387059183673, 'cm-245': 9.719835571283822e-07, 'cm-244': 1.768463308107458e-05, 'cm-246': 5.5208994767695116e-08, 'cm-247': 5.82141336423937e-10, 'eu-152': 2.852461424990223e-08, 'c